In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import pickle
pd.set_option('future.no_silent_downcasting', True)

In [3]:
# Carrega o dataset. Disponível em https://www.kaggle.com/datasets/sukhmandeepsinghbrar/housing-price-dataset
data = pd.read_csv('../datasets/house_price.csv')

# mostra as linhas e colunas do dataset
print(data.shape)

# visualiza as primeiras linhas do dataset
data.head()

(21613, 21)


,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7229300521,20141013T000000,231300.0,2,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21613 entries, 0 to 21612
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21613 non-null  int64  
 1   date           21613 non-null  object 
 2   price          21613 non-null  float64
 3   bedrooms       21613 non-null  int64  
 4   bathrooms      21613 non-null  float64
 5   sqft_living    21613 non-null  int64  
 6   sqft_lot       21613 non-null  int64  
 7   floors         21613 non-null  float64
 8   waterfront     21613 non-null  int64  
 9   view           21613 non-null  int64  
 10  condition      21613 non-null  int64  
 11  grade          21613 non-null  int64  
 12  sqft_above     21613 non-null  int64  
 13  sqft_basement  21613 non-null  int64  
 14  yr_built       21613 non-null  int64  
 15  yr_renovated   21613 non-null  int64  
 16  zipcode        21613 non-null  int64  
 17  lat            21613 non-null  float64
 18  long  

# Feature Engineer

In [5]:
cat_vars = ['waterfront', 'zipcode', 'view', 'condition', 'grade']
target = ['price']
num_vars = [ 
    'bedrooms',
    'bathrooms',
    'sqft_living',
    'sqft_lot',
    'floors',
    'sqft_above',
    'sqft_basement',
    'sqft_living15',
    'sqft_lot15'
    ]


### Tratamento de Tipos

In [6]:
for var in cat_vars:
    data[var] = data[var].astype(str)

In [7]:
for var in num_vars:
    # Apesar algumas vars terem sido interpretadas como float, se trata números inteiros.
    data[var] = data[var].astype(int)

In [8]:
data['price'] = data['price'].astype(float)

### Tratamento de Nulos

In [9]:
data[cat_vars] = data[cat_vars].fillna('99999')

In [10]:
#substituição dos missings pela
for var in num_vars:
    mean_val = data[var].mean()
    data[var] = data[var].fillna(mean_val)

### Variáveis temporais

In [11]:
data["tempo_imovel"] = pd.to_datetime("now").year - data['yr_built']
data["tempo_renovacao"] = np.where(
    data['yr_renovated'] > 0,
    pd.to_datetime("now").year - data['yr_renovated'],0)

In [12]:
exclude_vars = ['lat', 'long','yr_built','yr_renovated', 'date']
df = data.drop(exclude_vars, axis=1)

### Tratamento variáveis categóricas

In [13]:
for var in ['waterfront','view', 'condition', 'grade']:
    print(df[var].value_counts()/len(df))

waterfront
0    0.992458
1    0.007542
Name: count, dtype: float64
view
0    0.901726
2    0.044557
3    0.023597
1    0.015361
4    0.014760
Name: count, dtype: float64
condition
3    0.649193
4    0.262759
5    0.078703
2    0.007958
1    0.001388
Name: count, dtype: float64
grade
7     0.415537
8     0.280757
9     0.120992
6     0.094295
10    0.052468
11    0.018461
5     0.011197
12    0.004164
4     0.001342
13    0.000601
3     0.000139
1     0.000046
Name: count, dtype: float64


In [14]:
def apply_ordinal_encode(rare_perc, cat_vars_valid):
    # Vamos atribuir uma classe única a todas categorias com uma baixa frequência
    for var in cat_vars_valid:

        tmp = df.groupby(var)[var].count() / len(df)
            
        frequent_ls = tmp[tmp > rare_perc].index

        # substitui as categorias de baixa frequencia pela categoria "00000"
        df[var] = np.where(df[var].isin(frequent_ls), df[var], '00000')

        # Aplicar o encoder para a coluna
        encoder = OrdinalEncoder(categories=[df[var].unique()])

        df[f'{var}_encoded'] = encoder.fit_transform(df[[var]])

In [15]:
apply_ordinal_encode(0.01, ['zipcode', 'view', 'condition', 'grade'])

In [16]:
for var in ['zipcode','view', 'condition', 'grade']:
    tmp = df[[var, f'{var}_encoded']].drop_duplicates()
    print(tmp.set_index(var)[f'{var}_encoded'].to_dict())

{'98178': 0.0, '98125': 1.0, '98028': 2.0, '98136': 3.0, '98074': 4.0, '98053': 5.0, '98003': 6.0, '98198': 7.0, '98146': 8.0, '98038': 9.0, '00000': 10.0, '98115': 11.0, '98107': 12.0, '98126': 13.0, '98103': 14.0, '98133': 15.0, '98040': 16.0, '98092': 17.0, '98030': 18.0, '98112': 19.0, '98052': 20.0, '98027': 21.0, '98117': 22.0, '98058': 23.0, '98001': 24.0, '98056': 25.0, '98166': 26.0, '98023': 27.0, '98105': 28.0, '98042': 29.0, '98008': 30.0, '98059': 31.0, '98122': 32.0, '98144': 33.0, '98004': 34.0, '98034': 35.0, '98075': 36.0, '98116': 37.0, '98118': 38.0, '98199': 39.0, '98045': 40.0, '98168': 41.0, '98177': 42.0, '98065': 43.0, '98029': 44.0, '98006': 45.0, '98022': 46.0, '98033': 47.0, '98155': 48.0, '98031': 49.0, '98106': 50.0, '98072': 51.0, '98055': 52.0}
{'0': 0.0, '3': 1.0, '4': 2.0, '2': 3.0, '1': 4.0}
{'3': 0.0, '5': 1.0, '4': 2.0, '00000': 3.0}
{'7': 0.0, '6': 1.0, '8': 2.0, '11': 3.0, '9': 4.0, '5': 5.0, '10': 6.0, '00000': 7.0}


In [17]:
map_dict = {'3': 0.0, '5': 1.0, '4': 2.0, '00000': 3.0}

variable = 'condition'

print(df[variable].unique())

print(df[variable].map(map_dict).unique())

['3' '5' '4' '00000']
[0. 1. 2. 3.]


### Feature Scaling

In [18]:
robust_scaler = MinMaxScaler()
scaler_vars = ['sqft_living', 'sqft_lot', 'sqft_above', 'sqft_basement', 'sqft_living15', 'sqft_lot15']
df_num = pd.DataFrame(                       
    robust_scaler.fit_transform(df[scaler_vars]), 
    index=df.index,
    columns=scaler_vars
)

In [19]:
df_modelo =pd.concat([
    df_num,
    df[['bedrooms','bathrooms','floors','view_encoded','condition_encoded','grade_encoded', 'tempo_imovel', 'tempo_renovacao', 'zipcode_encoded','price']]
    ],axis=1)
df_modelo.head()

,sqft_living,sqft_lot,sqft_above,sqft_basement,sqft_living15,sqft_lot15,bedrooms,bathrooms,floors,view_encoded,condition_encoded,grade_encoded,tempo_imovel,tempo_renovacao,zipcode_encoded,price
0,0.067170,0.003108,0.097588,0.000000,0.161934,0.005742,2,1,1,0.0,0.0,0.0,70,0,0.0,231300.0
1,0.172075,0.004072,0.206140,0.082988,0.222165,0.008027,3,2,2,0.0,0.0,0.0,74,34,1.0,538000.0
2,0.036226,0.005743,0.052632,0.000000,0.399415,0.008513,2,1,1,0.0,0.0,1.0,92,0,2.0,180000.0
3,0.126038,0.002714,0.083333,0.188797,0.165376,0.004996,4,3,1,0.0,1.0,0.0,60,0,3.0,604000.0
4,0.104906,0.004579,0.152412,0.000000,0.241094,0.007871,3,2,1,0.0,0.0,2.0,38,0,4.0,510000.0


In [20]:
df_modelo.columns

Index(['sqft_living', 'sqft_lot', 'sqft_above', 'sqft_basement',
       'sqft_living15', 'sqft_lot15', 'bedrooms', 'bathrooms', 'floors',
       'view_encoded', 'condition_encoded', 'grade_encoded', 'tempo_imovel',
       'tempo_renovacao', 'zipcode_encoded', 'price'],
      dtype='object')

# Training

In [21]:
X_train, X_test, y_train, y_test = train_test_split(
    df_modelo.drop(['price'], axis=1), 
    df_modelo['price'], 
    test_size=0.20, 
    random_state=0,
)
X_test.to_csv("../datasets/house_price_predict_prepared.csv")

In [22]:
y_train = np.log(y_train)
y_test = np.log(y_test)

In [23]:
clf = RandomForestRegressor()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [24]:
clf

RandomForestRegressor()

In [25]:
print(
    mean_squared_error(y_test, y_pred),
    r2_score(y_test, y_pred)
    )

0.052309799151850564 0.804497484851962


In [26]:
if  r2_score(y_test, y_pred) >= 0.8 :
    dt = datetime.now().date()
    filename = f'../models/house_price_regression_{dt}.pkl'
    pickle.dump(clf, open(filename, 'wb'))